In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pydub import AudioSegment

In [4]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, LSTM)
from keras.models import Model, load_model
from PIL import Image
import numpy as np
import math

In [5]:
genres_musicaux = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']

In [6]:
def get_features():
  labels = []
  features = []
  i = 0
  #sample_rate = 22050
  #samples_per_segment = int(sample_rate*30/num_segment)

  for g in genres_musicaux:

    print(g)
    for filename in os.listdir(os.path.join('/content/gdrive/MyDrive/Projet Majeure/Premier test/audio3sec/',f'{g}')):
      if not filename.startswith('.'):    #avoiding the .DS_Store file
        path = os.path.join('/content/gdrive/MyDrive/Projet Majeure/Premier test/audio3sec/',f'{g}',f'{filename}')
        y,sr = librosa.load(path,duration = 3)
        data = np.array(librosa.feature.mfcc(y=y, sr=sr))
        if len(data[1]) != 130 :      #avoiding some problem of array length
          diff = 130-len(data[1])
          zeros = np.zeros((20,diff))
          data = np.concatenate((data,zeros),axis = 1)
          print(data.shape)
        features.append(data)
        labels.append(i)
    i += 1  
  return(np.array(features), labels)

In [49]:
X,y = get_features()

blues
classical
(20, 130)
country
(20, 130)
disco
(20, 130)
hiphop
(20, 130)
jazz
metal
pop
reggae
rock


In [50]:
print(X.shape)

(9900, 20, 130)


In [51]:
y = np.array(y)

In [52]:
#Split twice to get the validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=123)
#Print the shapes
X_train.shape, X_test.shape, X_val.shape, len(y_train), len(y_test), len(y_val)

((6336, 20, 130), (1980, 20, 130), (1584, 20, 130), 6336, 1980, 1584)

In [53]:
input_shape=(20,130)
# build network topology
model = keras.Sequential()

# 2 LSTM layers
model.add(keras.layers.LSTM(256, input_shape=input_shape, return_sequences=True))
model.add(keras.layers.LSTM(256))

# dense layer
model.add(Dense(256,activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(Dense(48, activation='relu'))
model.add(Dropout(0.4)) 
# output layer
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 20, 256)           396288    
                                                                 
 lstm_13 (LSTM)              (None, 256)               525312    
                                                                 
 dense_30 (Dense)            (None, 256)               65792     
                                                                 
 dense_31 (Dense)            (None, 128)               32896     
                                                                 
 dense_32 (Dense)            (None, 64)                8256      
                                                                 
 dropout_12 (Dropout)        (None, 64)                0         
                                                                 
 dense_33 (Dense)            (None, 48)               

In [54]:
model.compile(optimizer='adam',loss='SparseCategoricalCrossentropy',metrics=['acc'])

In [55]:
history = model.fit(X_train, y_train, epochs=30, batch_size=72, 
                    validation_data=(X_val, y_val), shuffle=False)

Epoch 1/30
88/88 [==============================] - 32s 322ms/step - loss: 2.1732 - acc: 0.1927 - val_loss: 1.9935 - val_acc: 0.2746
Epoch 2/30
88/88 [==============================] - 27s 308ms/step - loss: 1.9701 - acc: 0.2824 - val_loss: 1.8673 - val_acc: 0.3194
Epoch 3/30
88/88 [==============================] - 27s 307ms/step - loss: 1.8678 - acc: 0.3221 - val_loss: 1.8442 - val_acc: 0.3207
Epoch 4/30
88/88 [==============================] - 27s 309ms/step - loss: 1.7752 - acc: 0.3562 - val_loss: 1.8331 - val_acc: 0.3497
Epoch 5/30
88/88 [==============================] - 29s 331ms/step - loss: 1.6986 - acc: 0.3846 - val_loss: 1.8146 - val_acc: 0.3542
Epoch 6/30
88/88 [==============================] - 27s 306ms/step - loss: 1.5599 - acc: 0.4228 - val_loss: 1.7955 - val_acc: 0.3422
Epoch 7/30
88/88 [==============================] - 27s 307ms/step - loss: 1.4575 - acc: 0.4743 - val_loss: 1.8410 - val_acc: 0.3561
Epoch 8/30
88/88 [==============================] - 27s 308ms/step - 

In [56]:
y_pred=model.predict(X_test)
print(y_pred)
print('Confusion_matrix: ',tensorflow.math.confusion_matrix(y_test, np.argmax(y_pred,axis=1)))

62/62 [==============================] - 5s 62ms/step
[[6.7205178e-08 5.4535724e-07 2.6308738e-08 ... 1.6744925e-04
  3.5161974e-03 3.3205009e-08]
 [4.3955064e-15 7.8698265e-14 4.3389528e-09 ... 9.9995762e-01
  2.3652012e-06 6.2252507e-06]
 [9.9500811e-01 1.7868027e-11 4.4059372e-03 ... 5.8068546e-12
  4.9090527e-06 5.1965448e-04]
 ...
 [4.3674150e-01 5.0625295e-05 5.2037221e-01 ... 3.0165093e-05
  1.9644372e-02 9.8445741e-03]
 [4.2679966e-07 1.3623055e-13 8.9621410e-10 ... 5.6491672e-08
  4.7950261e-06 2.4324031e-06]
 [4.4677599e-09 1.0145205e-08 2.1728147e-06 ... 9.8953104e-01
  4.3464539e-04 6.0484628e-04]]
Confusion_matrix:  tf.Tensor(
[[ 56   1  21  21   9  11  16   1  23  39]
 [  5 112  12   0   0  50   2   3   4  10]
 [ 50   7  46   9  10  18   8  18  14  18]
 [ 25   0   4  39  20   8  25  24  20  33]
 [ 11   1   5  16  93   8  10  16  33   5]
 [ 33  20  17  12   6  52   3  14  15  26]
 [ 12   1   0  19   7   4 114   0   2  39]
 [  4   4   8   8  30   9   0 106  19  10]
 [ 20   

In [57]:
StrtMin = 1
StrtSec = 0

EndMin = 1
EndSec = 40

# Time to milliseconds conversion
StrtTime = StrtMin*60*1000+StrtSec*1000
EndTime = StrtMin*60*1000+EndSec*1000

In [66]:
audio = AudioSegment.from_wav('/content/Dr. Dre - Still D.R.E. ft. Snoop Dogg-_CL6n0FJZpk.wav')
newAudio = audio[StrtTime:EndTime]
newAudio.export('/content/song.wav', format='wav')

for w in range(0,10):
          t1 = 3*(w)*1000
          t2 = 3*(w+1)*1000
          newAudio = AudioSegment.from_wav('/content/song.wav')
          new = newAudio[t1:t2]
          new.export(f'/content/{str(w)}.wav', format='wav')


In [68]:
classe = []

for i in range(0,10):
  y,sr = librosa.load(f'/content/{str(w)}.wav',duration=3)
  data = np.array([librosa.feature.mfcc(y=y, sr=sr)])
  print(data.shape)

  prediction = model.predict(data)
  class_label = np.argmax(prediction)
  print(class_label)
  classe.append(class_label)

moyenne = int(sum(classe)/len(classe))
print(genres_musicaux[moyenne])

(1, 20, 130)
1/1 [==============================] - 0s 36ms/step
3
(1, 20, 130)
1/1 [==============================] - 0s 36ms/step
3
(1, 20, 130)
1/1 [==============================] - 0s 42ms/step
3
(1, 20, 130)
1/1 [==============================] - 0s 45ms/step
3
(1, 20, 130)
1/1 [==============================] - 0s 36ms/step
3
(1, 20, 130)
1/1 [==============================] - 0s 37ms/step
3
(1, 20, 130)
1/1 [==============================] - 0s 33ms/step
3
(1, 20, 130)
1/1 [==============================] - 0s 38ms/step
3
(1, 20, 130)
1/1 [==============================] - 0s 34ms/step
3
(1, 20, 130)
1/1 [==============================] - 0s 64ms/step
3
disco


In [69]:
prediction = model.predict(data)

1/1 [==============================] - 0s 26ms/step


In [70]:
class_label = np.argmax(prediction)
print(genres_musicaux[class_label])

disco
